In [2]:
import requests
from bs4 import BeautifulSoup
from scrapy.selector import Selector
import re
from time import sleep
import os
from selenium import webdriver
import pandas as pd
import numpy as np

In [2]:
chromedriver = './chromedriver/chromedriver.exe'
os.environ['webdriver.chrome.driver'] = chromedriver

In [3]:
search_term = 'https://www.mycareersfuture.sg/search?search=data warehouse&employmentType=Full Time&page='

In [ ]:
# initialise dataframe to hold the scrapping results
data_df = pd.DataFrame()

# to iterate through the pages of search results
for i in range(0,2):
    # concatenate the page number to get complete url
    url = search_term + str(i)
    
    # use chromedriver to get the results page
    driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver.exe")
    driver.get(url)
    sleep(2)
    html = driver.page_source
    
    results_soup = BeautifulSoup(html, 'lxml')
    
    # Get the links of job postings on the results page for the job listings
    class_list = results_soup.find_all('a', {'class': 'bg-white mb3 w-100 dib v-top pa3 no-underline flex-ns flex-wrap JobCard__card___22xP3'})
    link_list = []    # initialise list to hold the links of job postings

    for text in class_list:
        link_list.append('https://www.mycareersfuture.sg' + text.get('href'))
    
    print('==== List of job postings url ====')
    print(link_list)
    print('='*20)
    driver.close
    
    # iterate through the link_list to load each job posting
    for link in link_list:
        driver_job = webdriver.Chrome(executable_path="./chromedriver/chromedriver.exe")
        driver_job.get(link)
        sleep(1)
        posting = driver_job.page_source
        print('=== job posting ===')
        print(link)
        
        job_soup = BeautifulSoup(posting, 'lxml')
                
        company = job_soup.find('p', {'id':'company'}).get_text()
        job_title = job_soup.find('h1', {'id':'job_title'}).get_text()
        if job_soup.find('p', {'id':'address'}) != None:
            location = job_soup.find('p', {'id':'address'}).get_text()
        else:
            location = ''

        # getting the salary information
        salary_range = job_soup.find_all('div', {'class':'lh-solid'})
        for element in salary_range:
            salary = element.get_text()
        salary_type = job_soup.find('span', {'class':'salary_type dib f5 fw4 black-60 pr1 i pb'}).get_text()

        # getting the job description
        job_desc_section = job_soup.find('div', {'id':'job_description'})
        job_desc = job_desc_section.find('div', {'id':'content'}).get_text()

        # getting the job requirements
        reqmt_section = job_soup.find('div', {'id':'requirements'})
        if reqmt_section != None:
            job_reqmt = reqmt_section.find('div', {'id':'content'}).get_text()
        else:
            job_reqmt = ''

        job_posting = [{
            'company' : company,
            'job_title' : job_title,
            'location' : location,
            'salary' : salary,
            'salary_type' : salary_type,
            'job_description' : job_desc,
            'job_requirement' : job_reqmt,
            'url' : link
        }]
        
        driver_job.close

        # append the job posting to dataframe
        temp_df = pd.DataFrame(job_posting)
        data_df = pd.concat([data_df, temp_df], axis=0)
        
print('***** Done *****')

In [5]:
data_df.shape

(33, 8)

In [6]:
data_df.to_csv('data_wh_ft.csv', encoding='utf-8')

Combine CSVs

In [3]:
df = pd.read_csv('mycareersfuture.csv')
df_k = pd.read_csv('mycareersfuture_master_Kelvin.csv')

In [4]:
df_k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 771 entries, 0 to 770
Data columns (total 12 columns):
Unnamed: 0        771 non-null int64
Company           771 non-null object
JobTitle          771 non-null object
SalaryLower       771 non-null object
SalaryUpper       771 non-null object
OfficeLocation    771 non-null object
EmploymentType    771 non-null object
Seniority         771 non-null object
JobCategories     771 non-null object
RoleDesc          771 non-null object
Requirements      771 non-null object
mcfURL            765 non-null object
dtypes: int64(1), object(11)
memory usage: 72.4+ KB


In [5]:
df_k = df_k[~df_k['mcfURL'].isnull()]

In [7]:
df_k.head()

,Unnamed: 0,Company,JobTitle,SalaryLower,SalaryUpper,OfficeLocation,EmploymentType,Seniority,JobCategories,RoleDesc,Requirements,mcfURL
0,0,SPAN CONSULTANCY SERVICES PTE. LTD.,Business Analyst,"$5,800","$7,800Monthly","MOUNTBATTEN CENTRE, 231 MOUNTBATTEN ROAD 397999",Permanent,Senior Executive,Information Technology,§ Should have extensive experience in Retail ...,§ Min 5-8 years of experience § Should posse...,https://www.mycareersfuture.sg/job/5ef259bb8f5...
1,1,FUTUREX TECHNOLOGIES PTE. LTD.,Business Analyst,"$5,000","$7,000Monthly","TRADEHUB 21, 18 BOON LAY WAY 609966",Contract,Professional,Information Technology,Should have extensive experience in Retail Le...,Requirement Document Functional Specification...,https://www.mycareersfuture.sg/job/2d8dd9bde21...
2,2,CHUBB ASIA PACIFIC PTE. LTD.,Senior Business Analyst,"$108,000","$120,000Annually","CAPITAGREEN, 138 MARKET STREET 048946","Permanent, Contract",Executive,Professional Services,"Position Objective As Business Analyst, you wi...",Required Skills and Abilities Comprehensive wr...,https://www.mycareersfuture.sg/job/9251f72bf1e...
3,3,AXA INSURANCE PTE. LTD.,Senior Business Analyst (12 months Contract),"$4,000","$8,000Monthly","AXA TOWER, 8 SHENTON WAY 068811","Temporary, Contract, Full Time",Senior Executive,"Banking and Finance, Insurance",The AXA Group is a worldwide leader in insuran...,Successful qualifier of this mission: Univers...,https://www.mycareersfuture.sg/job/f5d8acc78b2...
4,4,APPLE SOUTH ASIA PTE. LTD.,"Senior Business Analyst, Asia Pacific Supply C...","$5,000","$10,000Monthly",7 ANG MO KIO STREET 64 569086,Full Time,Executive,Logistics / Supply Chain,The World Wide Supply Chain team is seeking to...,Technical person with the ability to quickly u...,https://www.mycareersfuture.sg/job/e9ca213819d...


In [13]:
for file in file_list:
    df_temp = pd.read_csv(file)
    df = pd.concat([df, df_temp], axis=0)

df.shape

(605, 9)

In [14]:
df = df.drop_duplicates(subset='url')

In [15]:
df.shape

(431, 9)

In [16]:
df.to_csv('mycareersfuture1.csv')